In [1]:
from IPython.core.display import display, HTML
display(HTML("<style>.container { width:100% !important; }</style>"))

In [2]:
import pandas as pd
import numpy as np
import os
import matplotlib.pyplot as plt
%matplotlib inline 
from sklearn.ensemble import RandomForestRegressor
from sklearn.tree import DecisionTreeRegressor
from sklearn.tree import export_graphviz
import matplotlib.pyplot as plt
import seaborn as sns

import lightgbm as lgbm
import gc
import xgboost as xgb

In [3]:
items           = pd.read_csv('items.csv')
item_categories = pd.read_csv('item_categories.csv')
shops           = pd.read_csv('shops.csv')
sales_train     = pd.read_csv('sales_train.csv.gz')
test            = pd.read_csv('test.csv.gz')

In [4]:
transactions = sales_train
transactions[['day','month', 'year']] = transactions['date'].str.split('.', expand=True).astype(int)


In [5]:
transactions = transactions.set_index('item_id').join(items.set_index('item_id'))
transactions.reset_index(inplace=True)
transactions = transactions[transactions['year'] != 2013]
transactions['y'] = transactions.groupby(['date_block_num', 'item_id', 'shop_id'])['item_cnt_day'].transform('sum').clip(0,20)


In [6]:
print(len(transactions))
transactions.head()

1668287


,item_id,date,date_block_num,shop_id,item_price,item_cnt_day,day,month,year,item_name,item_category_id,y
0,0,01.09.2014,20,54,58.0,1.0,1,9,2014,! ВО ВЛАСТИ НАВАЖДЕНИЯ (ПЛАСТ.) D,40,1.0
1,1,04.04.2014,15,55,4490.0,1.0,4,4,2014,!ABBYY FineReader 12 Professional Edition Full...,76,2.0
2,1,02.04.2014,15,55,4490.0,1.0,2,4,2014,!ABBYY FineReader 12 Professional Edition Full...,76,2.0
3,1,06.07.2014,18,55,4490.0,1.0,6,7,2014,!ABBYY FineReader 12 Professional Edition Full...,76,1.0
4,1,04.08.2014,19,55,4490.0,1.0,4,8,2014,!ABBYY FineReader 12 Professional Edition Full...,76,1.0


In [7]:
len(transactions)

1668287

In [8]:
transactions['gross'] = transactions['item_price'] * transactions['item_cnt_day']

In [9]:
number_of_items = transactions['item_id'].nunique()
print("number_of_items:", number_of_items)
number_of_categories = transactions['item_category_id'].nunique()
print("number_of_categories:", number_of_categories)
number_of_shops = transactions['shop_id'].nunique()
print("number_of_shops:", number_of_shops)
number_of_days = 365 + 365 - 30 - 31
print("number_of_days:", number_of_days)
number_of_blocks = transactions['date_block_num'].nunique()
print("number_of_blocks:", number_of_blocks)
total_sales = transactions['item_cnt_day'].sum()
print("total_sales:", total_sales)
total_gross = transactions['gross'].sum()
print("total_gross:", total_gross)
average_price = transactions['item_price'].mean()
print("average_price:", average_price)

number_of_items: 17054
number_of_categories: 79
number_of_shops: 55
number_of_days: 669
number_of_blocks: 22
total_sales: 2085473.0
total_gross: 2181401610.59
average_price: 1015.50230738


#ITEM

-UNITS
item_units
item_block_units
item_mean_units_block
item_day_units
item_mean_units_day
item_max_units_block
item_min_units_block
item_max_units_day
item_min_units_day

-TURNOVER
item_turnover
item_block_turnover
item_mean_turnover_block
item_day_turnover
item_mean_turnover_day
item_max_turnover_block
item_min_turnover_block
item_max_turnover_day
item_min_turnover_day


-TIME
item_days_of_activity
item_blocks_of_activity
item_mean_day_between_activity
item_longest_stretch_days_without_activity
item_longest_stretch_blocks_without_activity
item_longest_stretch_block_with_activity
(item_longest_stretch_day_with_activity deal with week ends)
item_days_between_start_and_first_activity
item_blocks_between_start_and_first_activity
item_first_block
item_last_block
item_first_day
item_last_day
item_activity_on_all_blocks


-PRICE
item_mean_price
item_min_price
item_max_price
item_number_differen_prices
item_price_amplitude (%age min/max)
item_deviation_mean_category_price


-TREND
is_first_two_full_blocks (actually second/third to make sure we have a "full" block if this was a new release !!!!
is_last_two_blocks
item_first_two_blocks_units
item_last_two_blocks_units
item_fluctuation_units_first_last_blocks
item_first_two_blocks_mean_price
item_last_two_blocks_mean_price
item_fluctuation_price_first_last_blocks

-ENCODINGS
item_share_of_total_units
item_share_of_total_gross
item_share_of_category_units
item_share_of_category_gross

In [ ]:
#CATEGORIES

In [10]:
transactions['total_sales_units'] = transactions.groupby(['item_id'])['item_cnt_day'].transform(np.sum)
transactions['block_sales_units'] = transactions.groupby(['item_id','date_block_num'])['item_cnt_day'].transform(np.sum)

In [11]:
def get_number_of_days_since_start(day,month, year):
    days = 0
    if year == 2015:
        days = 365
    def is_even(num):
        return num % 2 == 0
    half_of_month = int(month/2)
    even = (30*half_of_month) + (31*half_of_month)
    if is_even(month):
        days = days + even - 30 - day
    else:
        days = days + even + day
    return days

transactions['number_of_days_since_beginning'] = transactions.apply(lambda row: get_number_of_days_since_start(row['day'],row['month'], row['year']),axis=1)

In [12]:
transactions['first_block_sale'] = transactions.groupby(['item_id'])['date_block_num'].transform(np.min)
transactions['last_block_sale'] = transactions.groupby(['item_id'])['date_block_num'].transform(np.max)
transactions['first_day_sale'] = transactions.groupby(['item_id'])['number_of_days_since_beginning'].transform(np.min)
transactions['last_day_sale'] = transactions.groupby(['item_id'])['number_of_days_since_beginning'].transform(np.max)
transactions['first_year_sale'] = transactions.groupby(['item_id'])['year'].transform(np.min)
transactions['last_year_sale'] = transactions.groupby(['item_id'])['year'].transform(np.max)

In [13]:
transactions['sold_two_years'] = transactions['last_year_sale'] > transactions['first_year_sale']

In [14]:
transactions['total_days_of_sales'] = transactions['last_day_sale'] - transactions['first_day_sale'] + 1
transactions['average_sales_units_day'] = transactions['total_sales_units'] / transactions['total_days_of_sales']

In [15]:
transactions['total_blocks_of_sales'] = transactions['last_block_sale'] - transactions['first_block_sale'] + 1
transactions['average_sales_units_block'] = transactions['total_sales_units'] / transactions['total_blocks_of_sales']

In [16]:
number_of_days_with_a_sale = transactions.groupby(['item_id', 'date'], as_index=False).first().groupby('item_id').size()

In [17]:
transactions['number_of_days_with_a_sale'] = transactions['item_id'].map(number_of_days_with_a_sale)

In [18]:
number_of_blocks_with_a_sale = transactions.groupby(['item_id', 'date_block_num'], as_index=False).first().groupby('item_id').size()
transactions['number_of_blocks_with_a_sale'] = transactions['item_id'].map(number_of_blocks_with_a_sale)

In [19]:
max_day_sale = transactions.groupby(['item_id', 'date'],as_index=False)['item_cnt_day'].sum().groupby(['item_id'])['item_cnt_day'].max()
transactions['max_day_sale'] = transactions['item_id'].map(max_day_sale)

min_day_sale = transactions.groupby(['item_id', 'date'],as_index=False)['item_cnt_day'].sum().groupby(['item_id'])['item_cnt_day'].min()
transactions['min_day_sale'] = transactions['item_id'].map(min_day_sale)


In [20]:
max_block_sale = transactions.groupby(['item_id', 'date_block_num'],as_index=False)['item_cnt_day'].sum().groupby(['item_id'])['item_cnt_day'].max()
transactions['max_block_sale'] = transactions['item_id'].map(max_block_sale)

min_block_sale = transactions.groupby(['item_id', 'date_block_num'],as_index=False)['item_cnt_day'].sum().groupby(['item_id'])['item_cnt_day'].min()
transactions['min_block_sale'] = transactions['item_id'].map(min_block_sale)


In [21]:
gc.collect()

def get_max_stretch_without_sales_days(days):
    days = np.unique(days)
    max_stretch = 0
    len_days = len(days)
    for index,day in enumerate(sorted(days)):
        if index == len_days - 1:
            return max_stretch
        next_day = days[index+1]
        stretch = next_day - day
        if stretch > max_stretch:
            max_stretch = stretch
            

        
max_stretch_without_sales = transactions.groupby(['item_id', 'date'], as_index=False).first().groupby(['item_id'])['number_of_days_since_beginning']\
                                    .apply(list).apply(lambda x: get_max_stretch_without_sales_days(x))

transactions['max_stretch_in_days_without_sales'] = transactions['item_id'].map(max_stretch_without_sales)

In [22]:
gc.collect()

def get_max_stretch_without_sales_block(blocks):
    blocks = np.unique(blocks)
    max_stretch = 0
    len_blocks = len(blocks)
    for index,block in enumerate(sorted(blocks)):
        if index == len_blocks - 1:
            return max_stretch
        next_block = blocks[index+1]
        stretch = next_block - block
        if stretch > max_stretch:
            max_stretch = stretch
            

        
max_stretch_without_sales_block = transactions.groupby(['item_id'])['date_block_num']\
                                    .apply(list).apply(lambda x: get_max_stretch_without_sales_block(x))

transactions['max_stretch_in_blocks_without_sales'] = transactions['item_id'].map(max_stretch_without_sales_block)

In [23]:
def get_following_pairs(days):
    days = np.unique(days)
    len_days = len(days)
    following = []
    for index,day in enumerate(sorted(days)):
        if index == len_days - 1:
            return following
        next_day = days[index+1]
        if next_day == day + 1:
            following.append([day, next_day])
        
assert(get_following_pairs([1,2,5,6,7,8,9,11,12,15]) == [[1, 2], [5, 6], [6, 7], [7, 8], [8, 9], [11, 12]])
assert(get_following_pairs([1,2,5,6,7,10]) == [[1, 2], [5, 6], [6, 7]])
assert(get_following_pairs([1,2,4,5,7,9,10]) == [[1, 2], [4, 5], [9,10]])
assert(get_following_pairs([1,2,4,5,7,9,10,11,12,15]) == [[1, 2], [4, 5], [9,10],[10,11],[11,12]])

In [24]:
def get_longest_stretch(following_pairs, n=1,new_n=1):
    #print("following_pairs", following_pairs, " n: ", n, " new_n: ", new_n)
    len_pairs = len(following_pairs)
    if len_pairs == 0:
        return 0
    if len_pairs == 1:
        if new_n > n:
            return new_n
        return n
    if following_pairs[1][0] == following_pairs[0][1]:
        new_n+=1
    else:
        if new_n > n:
            n=new_n
        new_n=1
    return get_longest_stretch(following_pairs[1:], n,new_n)


assert(get_longest_stretch([]) == 0)
assert(get_longest_stretch([[1, 2], [2,3], [3, 4], [4,5] ,[8,9], [11, 12]]) == 4)
assert(get_longest_stretch([[-1, 0],[1, 2], [2,3], [3, 4], [4,5] ,[8,9], [11, 12]]) == 4)
assert(get_longest_stretch([[1, 2], [4,5] ,[8,9], [9,10],[10, 11]]) == 3)
assert(get_longest_stretch([[1, 2], [4,5] ,[8,9], [9,10],[10, 11],[20, 21], [25,26]]) == 3)
assert(get_longest_stretch([[1, 2], [4,5] ,[8,9], [9,10],[10, 11],[14, 15], [15,16]]) == 3)
assert(get_longest_stretch([[1, 2], [4,5] ,[8,9], [9,10],[10, 11],[14, 15], [15,16],[18,19] ,[22,23], [23,24],[24, 25]]) == 3)
assert(get_longest_stretch([[1, 2], [4,5], [7, 8]]) == 1)
assert(get_longest_stretch([[1, 2], [5, 6], [6, 7], [7,8], [14, 15]]) == 3)
assert(get_longest_stretch([[1, 2], [5, 6], [6, 7], [7,8], [14, 15], [15, 16]]) == 3)
assert(get_longest_stretch([[1, 2], [5, 6], [6, 7], [7,8], [14, 15], [15, 16], [16, 17]]) == 3)
assert(get_longest_stretch([[1, 2], [5, 6], [6, 7], [14, 15], [15, 16], [16, 17]]) == 3)
assert(get_longest_stretch([[1, 2], [5, 6], [6, 7], [7,8], [14, 15], [20, 21], [21, 22], [22,23],[23,24]]) == 4)

In [25]:
gc.collect()

max_stretch_with_sales_days = transactions.groupby(['item_id', 'date'], as_index=False).first().groupby(['item_id'])['number_of_days_since_beginning']\
                                    .apply(list).apply(lambda x: get_longest_stretch(get_following_pairs(x)))

transactions['max_stretch_in_days_with_sales'] = transactions['item_id'].map(max_stretch_with_sales_days)

gc.collect()

max_stretch_with_sales_blocks = transactions.groupby(['item_id'])['date_block_num']\
                                    .apply(list).apply(lambda x: get_longest_stretch(get_following_pairs(x)))

transactions['max_stretch_in_blocks_with_sales'] = transactions['item_id'].map(max_stretch_with_sales_blocks)

In [26]:
def get_average_days_between_sales(days):
    days = sorted(np.unique(days))
    if len(days) == 0:
        return 9999
    if len(days) == 1:
        return 999
    return np.mean(np.ediff1d(days)) / len(days)

average_days_between_sales = transactions.groupby(['item_id', 'date'], as_index=False).first().groupby(['item_id'])['number_of_days_since_beginning']\
                                    .apply(list).apply(lambda x: get_average_days_between_sales(x))

transactions['average_days_between_sales'] = transactions['item_id'].map(average_days_between_sales)

In [27]:
transactions['possibly_released_during_period'] = ((transactions['max_stretch_in_blocks_without_sales'] > 8) & (transactions['number_of_days_with_a_sale'] > 200)).astype(bool)

In [28]:
transactions['share_of_total_sold'] = transactions['total_sales_units'] * 100 / total_sales 

transactions['item_gross'] = transactions.groupby('item_id')['gross'].transform(np.sum)
transactions['share_of_total_gross'] = transactions['item_gross'] * 100 / total_gross 

In [31]:
transactions['item_min_price'] = transactions.groupby('item_id')['item_price'].transform(np.min)
transactions['item_max_price'] = transactions.groupby('item_id')['item_price'].transform(np.max)
transactions['price_fluctuation'] = ((transactions['item_max_price'] - transactions['item_min_price'] ) / transactions['item_min_price']) * 100
transactions['number_of_different_prices'] = transactions.groupby('item_id')['item_price'].transform('nunique')
transactions['item_average_price'] = transactions.groupby('item_id')['item_price'].transform(np.mean)



In [32]:
transactions['category_average_price'] = transactions.groupby(['item_category_id'])['item_price'].transform(np.mean)
transactions['deviation_category_price'] =  ((transactions['item_average_price'] - transactions['category_average_price'] ) / transactions['category_average_price']) * 100

In [33]:
list(range(1,8))

[1, 2, 3, 4, 5, 6, 7]

In [34]:

def is_first_two_blocks(first_block_sale, block):
    return block in [first_block_sale, first_block_sale+1]
def is_last_two_blocks(last_block_sale, block):
    return block in [last_block_sale - 1, last_block_sale]


transactions['is_first_two_blocks'] = transactions.apply(lambda row: is_first_two_blocks(row['first_block_sale'], row['date_block_num']),axis=1)
transactions['is_last_two_blocks'] = transactions.apply(lambda row: is_last_two_blocks(row['last_block_sale'], row['date_block_num']),axis=1)

In [35]:
a = transactions.groupby(['item_id','is_first_two_blocks'], as_index=False)['item_price'].mean()
b = a[a['is_first_two_blocks'] == True].set_index('item_id').iloc[:,1]

transactions['item_average_price_first_two_blocks'] = transactions['item_id'].map(b)

c = transactions.groupby(['item_id','is_last_two_blocks'], as_index=False)['item_price'].mean()
d = c[c['is_last_two_blocks'] == True].set_index('item_id').iloc[:,1]

transactions['item_average_price_last_two_blocks'] = transactions['item_id'].map(d)

transactions['fluctuation_item_average_price_first_last_two_blocks'] =  ((transactions['item_average_price_first_two_blocks'] \
                - transactions['item_average_price_last_two_blocks'] ) / transactions['item_average_price_first_two_blocks']) * 100 * -1

In [36]:
a = transactions.groupby(['item_id','is_first_two_blocks'], as_index=False)['item_cnt_day'].sum()
b = a[a['is_first_two_blocks'] == True].set_index('item_id').iloc[:,1]

transactions['first_two_blocks_sales_units'] = transactions['item_id'].map(b)

c = transactions.groupby(['item_id','is_last_two_blocks'], as_index=False)['item_cnt_day'].sum()
d = c[c['is_last_two_blocks'] == True].set_index('item_id').iloc[:,1]

transactions['last_two_blocks_sales_units'] = transactions['item_id'].map(d)

In [37]:
transactions['number_of_blocks_between_target_and_first'] = 34 - transactions['first_block_sale']
transactions['fluctuation_first_last_two_blocks'] =  ((transactions['first_two_blocks_sales_units'] - transactions['last_two_blocks_sales_units'] ) / transactions['first_two_blocks_sales_units']) * 100 * -1

In [38]:
transactions['new_release'] = 34 - transactions['first_block_sale'] < 4

In [39]:
sub_cats = {}
for i in range(1,8):
    sub_cats[i] = "Accessories"
sub_cats[8] = "Tickets "
sub_cats[9] = "Delivery of goods"
for i in range(10,18):
    sub_cats[i] = "Consoles"
for i in range(18,25):
    sub_cats[i] = "Game for Consoles"
sub_cats[25] = "Accessories for Games"
sub_cats[26] = "Android Games"
sub_cats[27] = "MAC Games"
for i in range(28,32):
    sub_cats[i] = "PC Games"
for i in range(32,37):
    sub_cats[i] = "Payment Cards"
for i in range(37,40):
    sub_cats[i] = "Cinema - Blu-ray"
sub_cats[40] = "Cinema - DVD"
sub_cats[41] = "Cinema - Collectible"
for i in range(42,46):
    sub_cats[i] = "Audiobooks"
for i in range(46,55):
    sub_cats[i] = "Books"
for i in range(55,57):
    sub_cats[i] = "Music - CD"
sub_cats[57] = "Music - MP3"
sub_cats[58] = "Music - Vinyl"
sub_cats[59] = "Music - Music Video"
sub_cats[60] = "Music - Gift Edition"
for i in range(61,74):
    sub_cats[i] = "Gifts"
for i in range(73,79):
    sub_cats[i] = "Software"
sub_cats[79] = "Utility"
for i in range(80,84):
    sub_cats[i] = "Misc"

In [40]:
transactions['subcategory'] = transactions['item_category_id'].apply(lambda x: sub_cats[x])

In [41]:
transactions['video_game'] = transactions["item_category_id"].isin(list(range(18,32)))
transactions['gaming_old_gen'] = transactions["item_category_id"].isin([10,11,15,18,19,23])
transactions['gaming_new_gen'] = transactions["item_category_id"].isin([12,14,16,20,22,24])
transactions['pc_games'] = transactions["item_category_id"].isin(list(range(27,32)))
transactions['payment_cards'] = transactions["item_category_id"].isin(list(range(32,37)))
transactions['movies'] = transactions["item_category_id"].isin(list(range(37,42)))
transactions['movies_niche'] = transactions["item_category_id"].isin([38,39])
transactions['books'] = transactions["item_category_id"].isin([42,55])
transactions['music'] = transactions["item_category_id"].isin(list(range(55,61)))
transactions['music_CD'] = transactions["item_category_id"].isin([55,56])
transactions['music_vinyl'] = transactions["item_category_id"].isin([58])
transactions['gifts'] = transactions["item_category_id"].isin(list(range(61,72)))
transactions['software'] = transactions["item_category_id"].isin(list(range(73,79)))

In [42]:
transactions['category_total_sales_units'] = transactions.groupby('item_category_id')['item_cnt_day'].transform(np.sum)
transactions['category_share_of_total_sold'] = transactions['category_total_sales_units'] * 100 / total_sales

transactions['category_gross'] = transactions.groupby('item_category_id')['gross'].transform(np.sum)
transactions['category_share_of_total_gross'] = transactions['category_gross'] / total_gross * 100

In [43]:
transactions['subcategory_total_sales_units'] = transactions.groupby('subcategory')['item_cnt_day'].transform(np.sum)
transactions['subcategory_share_of_total_sold'] = transactions['subcategory_total_sales_units'] / total_sales * 100

transactions['subcategory_gross'] = transactions.groupby('subcategory')['gross'].transform(np.sum)
transactions['subcategory_share_of_total_gross'] = transactions['subcategory_gross'] / total_gross * 100

In [44]:
a = transactions.groupby(['item_category_id','is_first_two_blocks'], as_index=False)['item_cnt_day'].sum()
b = a[a['is_first_two_blocks'] == True].set_index('item_category_id').iloc[:,1]

transactions['first_two_blocks_category_sales_units'] = transactions['item_category_id'].map(b)

c = transactions.groupby(['item_category_id','is_last_two_blocks'], as_index=False)['item_cnt_day'].sum()
d = c[c['is_last_two_blocks'] == True].set_index('item_category_id').iloc[:,1]

transactions['last_two_blocks_category_sales_units'] = transactions['item_category_id'].map(d)

In [45]:
transactions['fluctuation_category_first_last_two_blocks'] =  ((transactions['first_two_blocks_category_sales_units'] - transactions['last_two_blocks_category_sales_units'] ) / transactions['first_two_blocks_category_sales_units']) * 100 * -1

In [46]:
a = transactions.groupby(['subcategory','is_first_two_blocks'], as_index=False)['item_cnt_day'].sum()
b = a[a['is_first_two_blocks'] == True].set_index('subcategory').iloc[:,1]

transactions['first_two_blocks_subcategory_sales_units'] = transactions['subcategory'].map(b)

c = transactions.groupby(['subcategory','is_last_two_blocks'], as_index=False)['item_cnt_day'].sum()
d = c[c['is_last_two_blocks'] == True].set_index('subcategory').iloc[:,1]

transactions['last_two_blocks_subcategory_sales_units'] = transactions['subcategory'].map(d)

In [47]:
transactions['fluctuation_subcategory_first_last_two_blocks'] =  ((transactions['first_two_blocks_subcategory_sales_units'] - transactions['last_two_blocks_subcategory_sales_units'] ) / transactions['first_two_blocks_subcategory_sales_units']) * 100 * -1

In [48]:
shop_areas = {}
for i in range(0,2):
    shop_areas[i] = "Yakutsk"
shop_areas[2] = "Adygea"
shop_areas[3] = "Balashikha"
shop_areas[4] = "Volga"
shop_areas[5] = "Vologda"
for i in range(6,9):
    shop_areas[i] = "Voronezh"
shop_areas[9] = "Outbound Trading"
for i in range(10,12):
    shop_areas[i] = "Zhukovsky"
shop_areas[12] = "Online store emergency"
for i in range(13,15):
    shop_areas[i] = "Kazan"
shop_areas[15] = "Kaluga"
shop_areas[16] = "Kolomna"
for i in range(17,19):
    shop_areas[i] = "Krasnoyarsk"
shop_areas[19] = "Kursk"
for i in range(20,33):
    shop_areas[i] = "Moscow"
shop_areas[33] = "Mytishchi"
for i in range(34,36):
    shop_areas[i] = "N.Novgorod"
for i in range(36,38):
    shop_areas[i] = "Novosibirsk"
shop_areas[38] = "Omsk"
for i in range(39,42):
    shop_areas[i] = "RostovNaDonu"
for i in range(42,44):
    shop_areas[i] = "St. Petersburg"
for i in range(44,46):
    shop_areas[i] = "Samara"
shop_areas[46] = "Sergiev Posad"
shop_areas[47] = "Surgut"
shop_areas[48] = "Tomsk"
for i in range(49,52):
    shop_areas[i] = "Tyumen TC"
for i in range(52,54):
    shop_areas[i] = "Ufa"
shop_areas[54] = "Khimki"
shop_areas[55] = "Digital warehouse"
shop_areas[56] = "Chekhov"
for i in range(57,59):
    shop_areas[i] = "Yakutsk"
shop_areas[59] = "Yaroslavl"

In [49]:
transactions['area'] = transactions['shop_id'].apply(lambda x: shop_areas[x])

In [50]:
shop_ids_TC = [1,2,13,14,16,23,24,26,28,31,37,38,42,43,44,46,50,54,58]
shop_ids_TRK = [3,33,39,40]
shop_ids_SEC = [7,34,36,47,48,49,56]
shop_ids_shopping_center = [4,5,8,15,17,18,19,27,29,30,32,41,45,51,53,59]
shop_ids_moscow = list(range(20,33))

In [51]:
transactions['shop_TC'] = transactions['shop_id'].isin(shop_ids_TC)
transactions['shop_TRK'] = transactions['shop_id'].isin(shop_ids_TRK)
transactions['shop_SEC'] = transactions['shop_id'].isin(shop_ids_SEC)
transactions['shop_shopping_center'] = transactions['shop_id'].isin(shop_ids_shopping_center)
transactions['shop_moscow'] = transactions['shop_id'].isin(shop_ids_moscow)

In [52]:
transactions['shop_total_sales_units'] = transactions.groupby(['shop_id'])['item_cnt_day'].transform(np.sum)
transactions['shop_block_sales_units'] = transactions.groupby(['shop_id','date_block_num'])['item_cnt_day'].transform(np.sum)

In [53]:
transactions['area_total_sales_units'] = transactions.groupby(['area'])['item_cnt_day'].transform(np.sum)
transactions['area_block_sales_units'] = transactions.groupby(['area','date_block_num'])['item_cnt_day'].transform(np.sum)

In [54]:
transactions['shop_share_of_total_sold'] = transactions['shop_total_sales_units'] * 100 / total_sales 

transactions['shop_gross'] = transactions.groupby('shop_id')['gross'].transform(np.sum)
transactions['shop_share_of_total_gross'] = transactions['shop_gross'] * 100 / total_gross 

In [55]:
transactions['area_share_of_total_sold'] = transactions['area_total_sales_units'] * 100 / total_sales 

transactions['area_gross'] = transactions.groupby('area')['gross'].transform(np.sum)
transactions['area_share_of_total_gross'] = transactions['area_gross'] * 100 / total_gross 

In [56]:
transactions['shop_average_price'] =  transactions.groupby('shop_id')['item_price'].transform(np.mean)
transactions['shop_price_fluctuation'] = ((transactions['shop_average_price'] - average_price ) / average_price) * 100



In [57]:
a = transactions.groupby(['shop_id','is_first_two_blocks'], as_index=False)['item_price'].mean()
b = a[a['is_first_two_blocks'] == True].set_index('shop_id').iloc[:,1]

transactions['shop_average_price_first_two_blocks'] = transactions['shop_id'].map(b)

c = transactions.groupby(['shop_id','is_last_two_blocks'], as_index=False)['item_price'].mean()
d = c[c['is_last_two_blocks'] == True].set_index('shop_id').iloc[:,1]

transactions['shop_average_price_last_two_blocks'] = transactions['shop_id'].map(d)

transactions['fluctuation_shop_average_price_first_last_two_blocks'] =  ((transactions['shop_average_price_first_two_blocks'] \
                - transactions['shop_average_price_last_two_blocks'] ) / transactions['shop_average_price_first_two_blocks']) * 100 * -1

In [58]:
day_mean = transactions.groupby(['shop_id','date'],as_index=False)['item_cnt_day'].sum().groupby('shop_id')['item_cnt_day'].mean()
transactions['shop_average_sales_units_day'] = transactions['shop_id'].map(day_mean)

day_gross = transactions.groupby(['shop_id','date'],as_index=False)['gross'].sum().groupby('shop_id')['gross'].mean()
transactions['shop_average_gross_day'] = transactions['shop_id'].map(day_gross)

block_mean = transactions.groupby(['shop_id','date_block_num'],as_index=False)['item_cnt_day'].sum().groupby('shop_id')['item_cnt_day'].mean()
transactions['shop_average_sales_units_block'] = transactions['shop_id'].map(block_mean)

block_gross = transactions.groupby(['shop_id','date_block_num'],as_index=False)['gross'].sum().groupby('shop_id')['gross'].mean()
transactions['shop_average_gross_block'] = transactions['shop_id'].map(block_gross)

In [59]:
transactions.groupby(['shop_id','date_block_num'],as_index=False)['item_cnt_day'].sum().groupby('shop_id')['item_cnt_day'].mean()

shop_id
2      939.045455
3      825.272727
4     1133.636364
5     1275.727273
6     2449.545455
7     1766.909091
9     3113.333333
10     637.666667
11     572.000000
12    2467.136364
13     779.250000
14    1195.181818
15    1769.227273
16    1430.090909
17    1226.384615
18    1450.272727
19    1850.136364
20    2936.000000
21    1970.681818
22    1515.090909
24    1634.000000
25    6334.000000
26    1667.000000
27    4111.550000
28    4813.636364
29    1633.470588
30    1951.214286
31    8104.090909
33     609.111111
34     403.187500
35    1861.318182
36     330.000000
37    1118.954545
38    1513.545455
39     830.850000
40     449.363636
41    1119.500000
42    4145.545455
43    1877.615385
44    1086.909091
45    1013.045455
46    1960.590909
47    2061.181818
48    1311.000000
49     767.863636
50    1562.681818
51    1075.714286
52    1257.227273
53    1621.681818
54    5590.937500
55    2543.409091
56    1977.090909
57    3679.272727
58    2252.818182
59    1212.954545
Na

In [60]:
print(transactions[(transactions['item_id'] == 3076) & (transactions['date_block_num'] == 12)]['item_price'].mean())
print(transactions[(transactions['item_id'] == 3076) & (transactions['date_block_num'] == 32)]['item_price'].mean())

710.218180077
1191.39705882


In [61]:
###
#DEBUG
###


pd.set_option('display.max_columns', None)  
pd.set_option('display.expand_frame_repr', False)
pd.set_option('max_colwidth', -1)
transactions.sample(10).sort_values(by=['total_sales_units'], ascending=False)
#transactions[transactions['item_category_id'] == 58].sample(10).sort_values(by=['total_sales_units'], ascending=False)

,item_id,date,date_block_num,shop_id,item_price,item_cnt_day,day,month,year,item_name,item_category_id,y,gross,total_sales_units,block_sales_units,number_of_days_since_beginning,first_block_sale,last_block_sale,first_day_sale,last_day_sale,first_year_sale,last_year_sale,sold_two_years,total_days_of_sales,average_sales_units_day,total_blocks_of_sales,average_sales_units_block,number_of_days_with_a_sale,number_of_blocks_with_a_sale,max_day_sale,min_day_sale,max_block_sale,min_block_sale,max_stretch_in_days_without_sales,max_stretch_in_blocks_without_sales,max_stretch_in_days_with_sales,max_stretch_in_blocks_with_sales,average_days_between_sales,possibly_released_during_period,share_of_total_sold,item_gross,share_of_total_gross,item_min_price,item_max_price,price_fluctuation,number_of_different_prices,item_average_price,category_average_price,deviation_category_price,is_first_two_blocks,is_last_two_blocks,item_average_price_first_two_blocks,item_average_price_last_two_blocks,fluctuation_item_average_price_first_last_two_blocks,first_two_blocks_sales_units,last_two_blocks_sales_units,number_of_blocks_between_target_and_first,fluctuation_first_last_two_blocks,new_release,subcategory,video_game,gaming_old_gen,gaming_new_gen,pc_games,payment_cards,movies,movies_niche,books,music,music_CD,music_vinyl,gifts,software,category_total_sales_units,category_share_of_total_sold,category_gross,category_share_of_total_gross,subcategory_total_sales_units,subcategory_share_of_total_sold,subcategory_gross,subcategory_share_of_total_gross,first_two_blocks_category_sales_units,last_two_blocks_category_sales_units,fluctuation_category_first_last_two_blocks,first_two_blocks_subcategory_sales_units,last_two_blocks_subcategory_sales_units,fluctuation_subcategory_first_last_two_blocks,area,shop_TC,shop_TRK,shop_SEC,shop_shopping_center,shop_moscow,shop_total_sales_units,shop_block_sales_units,area_total_sales_units,area_block_sales_units,shop_share_of_total_sold,shop_gross,shop_share_of_total_gross,area_share_of_total_sold,area_gross,area_share_of_total_gross,shop_average_price,shop_price_fluctuation,shop_average_price_first_two_blocks,shop_average_price_last_two_blocks,fluctuation_shop_average_price_first_last_two_blocks,shop_average_sales_units_day,shop_average_gross_day,shop_average_sales_units_block,shop_average_gross_block
199253,1855,03.01.2015,24,47,799.0,1.0,3,1,2015,"Battlefield 4 [PC, русская версия]",30,8.0,799.0,5985.0,336.0,368,12,33,1,638,2014,2015,True,638,9.380878,22,272.045455,478,21,129.0,1.0,1094.0,1.0,55,2,31,17,0.009315,False,0.286985,4912318.43,0.225191,0.50,1199.0,239700.000000,166.0,851.437050,542.437008,56.965148,False,False,871.808503,699.000000,-19.821842,2073.0,8.0,22,-99.614086,False,PC Games,True,False,False,True,False,False,False,False,False,False,False,False,False,221327.0,10.612796,1.368145e+08,6.271862,361094.0,17.314729,2.786370e+08,12.773300,73427.0,11674.0,-84.101216,138993.0,24513.0,-82.363860,Surgut,False,False,True,False,False,45346.0,2410.0,45346.0,2410.0,2.174375,5.515765e+07,2.528542,2.174375,5.515765e+07,2.528542,1226.199190,20.748046,1463.512282,1258.989796,-13.974771,67.985007,82695.123043,2061.181818,2.507166e+06
1310123,7856,30.07.2014,18,27,799.0,1.0,30,7,2014,World of Warcraft. Карта оплаты игрового времени (online) (рус.в.) (60 дней) (Jewel),28,7.0,799.0,4956.0,230.0,213,12,33,1,639,2014,2015,True,639,7.755869,22,225.272727,616,22,71.0,0.0,550.0,28.0,31,1,31,21,0.005439,False,0.237644,4053140.33,0.185804,531.24,1199.0,125.698366,37.0,824.928521,739.775866,11.510602,False,False,791.429927,1191.737864,50.580339,609.0,114.0,22,-81.280788,False,PC Games,True,False,False,True,False,False,False,False,False,False,False,False,False,97781.0,4.688673,8.081408e+07,3.704686,361094.0,17.314729,2.786370e+08,12.773300,47455.0,4892.0,-89.691286,138993.0,24513.0,-82.363860,Moscow,False,False,False,True,True,82231.0,3756.0,716036.0,29879.0,3.943038,9.355175e+07,4.288607,34.334465,7.135042e+08,32.708520,1134.321412,11.7005

In [62]:
print(transactions.columns.tolist())
len(transactions.columns)

['item_id', 'date', 'date_block_num', 'shop_id', 'item_price', 'item_cnt_day', 'day', 'month', 'year', 'item_name', 'item_category_id', 'y', 'gross', 'total_sales_units', 'block_sales_units', 'number_of_days_since_beginning', 'first_block_sale', 'last_block_sale', 'first_day_sale', 'last_day_sale', 'first_year_sale', 'last_year_sale', 'sold_two_years', 'total_days_of_sales', 'average_sales_units_day', 'total_blocks_of_sales', 'average_sales_units_block', 'number_of_days_with_a_sale', 'number_of_blocks_with_a_sale', 'max_day_sale', 'min_day_sale', 'max_block_sale', 'min_block_sale', 'max_stretch_in_days_without_sales', 'max_stretch_in_blocks_without_sales', 'max_stretch_in_days_with_sales', 'max_stretch_in_blocks_with_sales', 'average_days_between_sales', 'possibly_released_during_period', 'share_of_total_sold', 'item_gross', 'share_of_total_gross', 'item_min_price', 'item_max_price', 'price_fluctuation', 'number_of_different_prices', 'item_average_price', 'category_average_price', 'dev

112

In [63]:
print(shops.to_string())

                                          shop_name  shop_id
0   !Якутск Орджоникидзе, 56 фран                    0      
1   !Якутск ТЦ "Центральный" фран                    1      
2   Адыгея ТЦ "Мега"                                 2      
3   Балашиха ТРК "Октябрь-Киномир"                   3      
4   Волжский ТЦ "Волга Молл"                         4      
5   Вологда ТРЦ "Мармелад"                           5      
6   Воронеж (Плехановская, 13)                       6      
7   Воронеж ТРЦ "Максимир"                           7      
8   Воронеж ТРЦ Сити-Парк "Град"                     8      
9   Выездная Торговля                                9      
10  Жуковский ул. Чкалова 39м?                       10     
11  Жуковский ул. Чкалова 39м²                       11     
12  Интернет-магазин ЧС                              12     
13  Казань ТЦ "Бехетле"                              13     
14  Казань ТЦ "ПаркХаус" II                          14     
15  Калуга ТРЦ "XXI век"